In [2]:
# Decision_Trees

my_data=[['Wizz Air','UK','no',4,'Economy'],
        ['Lufthansa','France','yes',23,'1st Class'],
        ['TAROM','UK','yes',8,'Bussines '],
        ['TAROM','France','yes',12,'Bussines '],
        ['Lufthansa','UK','yes',21,'1st Class'],
        ['Blue Air','France','no',3,'Economy'],
        ['Blue Air','UK','no',4,'Bussines '],
        ['Lufthansa','UK','yes',24,'1st Class'],
        ['Wizz Air','France','no',1,'Economy'],
        ['TAROM','UK','no',1,'Economy'],
        ['Lufthansa','UK','no',3,'Economy'],
        ['Austian Airlines','Austria','no',1,'Economy'],
        ['TAROM','France','yes',8,'Bussines '],
        ['Wizz Air','UK','no',3,'Economy'],
        ['Lufthansa','UK','yes',10,'Bussines '],
        ['Austrian Airlines','Austria','yes',7,'Bussines ']]

# Impartim datele in functie de referinta acestora( Companie aeriana, Destinatie, Subscriprie Club discount, #de zboruri/an, Clasa)

def divideset(rows,column,value):
   # Facem o functie care ne spune daca este in primul grup (true) sau in al doilea (false)
   split_function=None
   if isinstance(value,int) or isinstance(value,float): 
      split_function=lambda row:row[column]>=value
   else:
      split_function=lambda row:row[column]==value
   
   # Impartim randurile in 2 grupe si returnam
   set1=[row for row in rows if split_function(row)]
   set2=[row for row in rows if not split_function(row)]
   return (set1,set2)

divideset(my_data,1,'yes')

divideset(my_data,3,20)

# Vom numara rezulatele posibile
def uniquecounts(rows):
   results={}
   for row in rows:
      # Rezultatul este ultima coloana
      r=row[len(row)-1]
      if r not in results: results[r]=0
      results[r]+=1
   return results

print(uniquecounts(my_data))

print(divideset(my_data,3,20)[0])
print(uniquecounts(divideset(my_data,3,20)[0]))
print("")
print(divideset(my_data,3,20)[1])
print(uniquecounts(divideset(my_data,3,20)[1]))

# Calculam entropia= suma dep(x)log(p(x)), din rezultate posibile
def entropy(rows):
   from math import log
   log2=lambda x:log(x)/log(2)  
   results=uniquecounts(rows)
   # Now calculate the entropy
   ent=0.0
   for r in results.keys():
      p=float(results[r])/len(rows)
      ent=ent-p*log2(p)
   return ent

set1,set2=divideset(my_data,3,20)
entropy(set1), entropy(set2)

entropy(my_data)



class decisionnode:
  def __init__(self,col=-1,value=None,results=None,tb=None,fb=None):
    self.col=col
    self.value=value
    self.results=results
    self.tb=tb
    self.fb=fb

def buildtree(rows,scoref=entropy): 
  if len(rows)==0: return decisionnode() #len(rows) este numarul de unitati
  current_score=scoref(rows)

# Introducem variabile pentru cel mai bun criteriu
  best_gain=0.0
  best_criteria=None
  best_sets=None
  
  column_count=len(rows[0])-1   #Numaram atributele/coloanele.
                               
  for col in range(0,column_count):
    # generam lista tuturor valorilor posibile
    global column_values        
    column_values={}            
    for row in rows:
       column_values[row[col]]=1   
    # Impartim randurile in functie de fiecare valoare din coloana
    for value in column_values.keys(): 
      (set1,set2)=divideset(rows,col,value) 
      
# Information gain
      p=float(len(set1))/len(rows) 
      gain=current_score-p*scoref(set1)-(1-p)*scoref(set2) #cnf. formulei information gain
      if gain>best_gain and len(set1)>0 and len(set2)>0: 
        best_gain=gain
        best_criteria=(col,value)
        best_sets=(set1,set2)
        
# Crearea frunzelor  
  if best_gain>0:
    trueBranch=buildtree(best_sets[0])
    falseBranch=buildtree(best_sets[1])
    return decisionnode(col=best_criteria[0],value=best_criteria[1],
                        tb=trueBranch,fb=falseBranch)
  else:
    return decisionnode(results=uniquecounts(rows))


tree=buildtree(my_data)


def printtree(tree,indent=''):
    if tree.results!=None:
        print(str(tree.results))
    else:
        print(str(tree.col)+':'+str(tree.value)+'? ')
        print indent+'T->',
        printtree(tree.tb,indent+'  ')
        print indent+'F->',
        printtree(tree.fb,indent+'  ')
        
printtree(tree)

def getwidth(tree):
  if tree.tb==None and tree.fb==None: return 1
  return getwidth(tree.tb)+getwidth(tree.fb)

def getdepth(tree):
  if tree.tb==None and tree.fb==None: return 0
  return max(getdepth(tree.tb),getdepth(tree.fb))+1


from PIL import Image,ImageDraw

def drawtree(tree,jpeg='tree.jpg'):
  w=getwidth(tree)*100
  h=getdepth(tree)*100+120

  img=Image.new('RGB',(w,h),(255,255,255))
  draw=ImageDraw.Draw(img)

  drawnode(draw,tree,w/2,20)
  img.save(jpeg,'JPEG')
  
def drawnode(draw,tree,x,y):
  if tree.results==None:
    
    w1=getwidth(tree.fb)*100
    w2=getwidth(tree.tb)*100

    left=x-(w1+w2)/2
    right=x+(w1+w2)/2

    draw.text((x-20,y-10),str(tree.col)+':'+str(tree.value),(0,0,0))

    draw.line((x,y,left+w1/2,y+100),fill=(255,0,0))
    draw.line((x,y,right-w2/2,y+100),fill=(255,0,0))

    drawnode(draw,tree.fb,left+w1/2,y+100)
    drawnode(draw,tree.tb,right-w2/2,y+100)
  else:
    txt=' \n'.join(['%s:%d'%v for v in tree.results.items()])
    draw.text((x-20,y),txt,(0,0,0))
    
drawtree(tree,jpeg='treeview.jpg')

def classify(observation,tree):
  if tree.results!=None:
    return tree.results
  else:
    v=observation[tree.col]
    branch=None
    if isinstance(v,int) or isinstance(v,float):
      if v>=tree.value: branch=tree.tb
      else: branch=tree.fb
    else:
      if v==tree.value: branch=tree.tb
      else: branch=tree.fb
    return classify(observation,branch)

classify(['TAROM','France','yes',11],tree)




{'Bussines ': 6, '1st Class': 3, 'Economy': 7}
[['Lufthansa', 'France', 'yes', 23, '1st Class'], ['Lufthansa', 'UK', 'yes', 21, '1st Class'], ['Lufthansa', 'UK', 'yes', 24, '1st Class']]
{'1st Class': 3}

[['Wizz Air', 'UK', 'no', 4, 'Economy'], ['TAROM', 'UK', 'yes', 8, 'Bussines '], ['TAROM', 'France', 'yes', 12, 'Bussines '], ['Blue Air', 'France', 'no', 3, 'Economy'], ['Blue Air', 'UK', 'no', 4, 'Bussines '], ['Wizz Air', 'France', 'no', 1, 'Economy'], ['TAROM', 'UK', 'no', 1, 'Economy'], ['Lufthansa', 'UK', 'no', 3, 'Economy'], ['Austian Airlines', 'Austria', 'no', 1, 'Economy'], ['TAROM', 'France', 'yes', 8, 'Bussines '], ['Wizz Air', 'UK', 'no', 3, 'Economy'], ['Lufthansa', 'UK', 'yes', 10, 'Bussines '], ['Austrian Airlines', 'Austria', 'yes', 7, 'Bussines ']]
{'Bussines ': 6, 'Economy': 7}
2:no? 
T-> 0:Blue Air? 
  T-> 1:UK? 
    T-> {'Bussines ': 1}
    F-> {'Economy': 1}
  F-> {'Economy': 6}
F-> 3:21? 
  T-> {'1st Class': 3}
  F-> {'Bussines ': 5}


{'Bussines ': 5}